In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("../Dataset/train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_val, y_train, y_val = train_test_split(data.drop("Survived", axis=1), data["Survived"], test_size=0.2, random_state=0)

In [6]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
140,141,3,"Boulos, Mrs. Joseph (Sultana)",female,NaN,0,2,2678,15.2458,NaN,C
439,440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,NaN,S
817,818,2,"Mallet, Mr. Albert",male,31.0,1,1,S.C./PARIS 2079,37.0042,NaN,C
378,379,3,"Betros, Mr. Tannous",male,20.0,0,0,2648,4.0125,NaN,C
491,492,3,"Windelov, Mr. Einar",male,21.0,0,0,SOTON/OQ 3101317,7.2500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
835,836,1,"Compton, Miss. Sara Rebecca",female,39.0,1,1,PC 17756,83.1583,E49,C
192,193,3,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,19.0,1,0,350046,7.8542,NaN,S
629,630,3,"O'Connell, Mr. Patrick D",male,NaN,0,0,334912,7.7333,NaN,Q
559,560,3,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,36.0,1,0,345572,17.4000,NaN,S


In [7]:
DropAndImpute = ColumnTransformer(
    transformers=[
        ("AgeImpute", SimpleImputer(strategy='mean', copy=False), [4]),
        ("EmbarkedImpute", SimpleImputer(strategy='most_frequent', copy=False), [10]),
        ("CabinImpute", SimpleImputer(strategy='constant', fill_value=0 ,add_indicator=True),[9]), 
    ], remainder="passthrough"
)

In [8]:
DropAndImpute.set_output(transform='pandas')

,transformers,"[('AgeImpute', ...), ('EmbarkedImpute', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [9]:
colTrans1 = DropAndImpute.fit_transform(X_train)
colTrans1

,AgeImpute__Age,EmbarkedImpute__Embarked,CabinImpute__Cabin,CabinImpute__missingindicator_Cabin,remainder__PassengerId,remainder__Pclass,remainder__Name,remainder__Sex,remainder__SibSp,remainder__Parch,remainder__Ticket,remainder__Fare
140,29.745184,C,0,True,141,3,"Boulos, Mrs. Joseph (Sultana)",female,0,2,2678,15.2458
439,31.000000,S,0,True,440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,0,0,C.A. 18723,10.5000
817,31.000000,C,0,True,818,2,"Mallet, Mr. Albert",male,1,1,S.C./PARIS 2079,37.0042
378,20.000000,C,0,True,379,3,"Betros, Mr. Tannous",male,0,0,2648,4.0125
491,21.000000,S,0,True,492,3,"Windelov, Mr. Einar",male,0,0,SOTON/OQ 3101317,7.2500
...,...,...,...,...,...,...,...,...,...,...,...,...
835,39.000000,C,E49,False,836,1,"Compton, Miss. Sara Rebecca",female,1,1,PC 17756,83.1583
192,19.000000,S,0,True,193,3,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,1,0,350046,7.8542
629,29.745184,Q,0,True,630,3,"O'Connell, Mr. Patrick D",male,0,0,334912,7.7333
559,36.000000,S,0,True,560,3,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,1,0,345572,17.4000


In [10]:
colTrans1.columns

Index(['AgeImpute__Age', 'EmbarkedImpute__Embarked', 'CabinImpute__Cabin',
       'CabinImpute__missingindicator_Cabin', 'remainder__PassengerId',
       'remainder__Pclass', 'remainder__Name', 'remainder__Sex',
       'remainder__SibSp', 'remainder__Parch', 'remainder__Ticket',
       'remainder__Fare'],
      dtype='object')

In [11]:
colTrans1["remainder__Name"].apply(lambda x: x.split(",")[1].split()[0])

140     Mrs.
439      Mr.
817      Mr.
378      Mr.
491      Mr.
       ...  
835    Miss.
192    Miss.
629      Mr.
559     Mrs.
684      Mr.
Name: remainder__Name, Length: 712, dtype: object

In [12]:
from pipelineHelpers import extractTitle, extractAgeCategory, famCategory, ticketCategory, cabinCategory

extractTitle(colTrans1["remainder__Name"])

,NameTitle
140,Mrs.
439,Mr.
817,Mr.
378,Mr.
491,Mr.
...,...
835,Miss.
192,Miss.
629,Mr.
559,Mrs.


In [13]:
extractAgeCategory(colTrans1["AgeImpute__Age"])

,AgeCategory
140,Adults
439,Adults
817,Adults
378,Adults
491,Adults
...,...
835,Middle
192,Adults
629,Adults
559,Middle


In [14]:
famCategory(colTrans1.iloc[:, 8:10])

,FamSize
140,2
439,0
817,2
378,0
491,0
...,...
835,2
192,1
629,0
559,1


In [15]:
ticketCategory(colTrans1["remainder__Ticket"])

,TicketCat
0,NA
1,C.A.
2,S.C./PARIS
3,NA
4,SOTON/OQ
...,...
707,PC
708,NA
709,NA
710,NA


In [16]:
ticketCategory(colTrans1["remainder__Ticket"]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   TicketCat  712 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [17]:
cabinCategory(colTrans1["CabinImpute__Cabin"])

,CabinCategory
0,MISSING
1,MISSING
2,MISSING
3,MISSING
4,MISSING
...,...
707,E
708,MISSING
709,MISSING
710,MISSING


In [18]:
cabinCategory(colTrans1["CabinImpute__Cabin"]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CabinCategory  712 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [19]:
CategoriesConstruct = ColumnTransformer(
    transformers= [
        ("TitleExtract", extractTitle, [6]),
        ("ExtractAgeCategory", extractAgeCategory, [0]),
        ("FamCategory", famCategory, [8,9]),
        ("TicketCategory", ticketCategory, [10]),
        ("CabinCategory", cabinCategory, [2]),
    ], remainder='drop'
)

In [20]:
pipe = Pipeline(
    ("DropAndImpute", DropAndImpute)
)